Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
# 1. Импорт библиотек

import numpy as np
import pandas as pd

import random
import re

import torch
import transformers
import nltk

from tqdm import notebook
from pymystem3 import Mystem
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score

import warnings


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1. Импортировал все необходимые библиотеки

In [2]:
#2. Загрузка файла

df_tweets = pd.read_csv('/datasets/toxic_comments.csv')

## 2.1. Установил ширину столбца, чтобы была возможность просмотреть твиты
pd.set_option('display.max_colwidth',200)

1. Загрузил файл
2. Установил ширину столбцов

In [3]:
# 3. Визуальный осмотр файлов

print (df_tweets.shape)
print (df_tweets.info())
display (df_tweets.describe())
display (df_tweets.head(15))

(159571, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about...",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tid...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encycl...",0
9,alignment on this subject and which are contrary to those of DuLithgow,0


1. Есть очень длинные записи

In [3]:
# 4. Функция по избавлению от длинных твитов (для Berta)

def length (text):
        if len (text) < 300:
            return 0
        else:
            return 1
df_tweets['length'] = df_tweets['text'].apply(length)
df_tweets = df_tweets.query('length == 0')
print (df_tweets.info())
#df_tweets = df_tweets.sample(n = 600)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100776 entries, 0 to 159570
Data columns (total 3 columns):
text      100776 non-null object
toxic     100776 non-null int64
length    100776 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.1+ MB
None


1. Создал функцию по избавлению от длинных твитов. Bert больше 512 символов не переваривает + для скорости работы (все виснет)
2. Также создал возможность делать сэмплы

# 2. Обучение

In [8]:
# 5. Лемматизация

lemmatizer = WordNetLemmatizer()

def lemmas (text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text) 
    text = " ".join(text.split())
    
    word_list = nltk.word_tokenize(text)

    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output
df_tweets['lemmas'] = df_tweets['text'].apply(lemmas)

display (df_tweets['lemmas'].head(30))

0     Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalism just closure on some GAs after I voted at New York Dolls FAC And please don t remove th...
1                                                                                                                      D aww He match this background colour I m seemingly stuck with Thanks talk January UTC
2     Hey man I m really not trying to edit war It s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page He seems to care more about th...
4                                                                                                                                             You sir are my hero Any chance you remember what page that s on
5                                                                                                                                                       Congratulations from me 

In [9]:
# Тренировка - убираю слова меньше 3-х символов

def lem (lemmas):
    word_list = nltk.word_tokenize(lemmas)
    lemmatized_output = " ".join(w for w in word_list if len(w) > 3) 
    return lemmatized_output

df_tweets['lemmas_1'] = df_tweets['lemmas'].apply(lem)


display (df_tweets['lemmas_1'].head(3))

0    Explanation edits made under username Hardcore Metallica were reverted They weren vandalism just closure some after voted York Dolls please remove template from talk page since retired
1                                                                                                                       match this background colour seemingly stuck with Thanks talk January
2                        really trying edit just that this constantly removing relevant information talking through edits instead talk page seems care more about formatting than actual info
Name: lemmas_1, dtype: object

1. Лемматизирую наши твиты
2. Для лемматизации использую WorldNetLemmatizer. Mysystem, который был в тренажере не подходит, так как оказывается работает только с русскими фразами

In [10]:
# 6. Разделяю выборки, очищаю от стоп-слов, обучаю tf_idf

features_train, features_test, target_train, target_test = train_test_split(df_tweets['lemmas_1'],df_tweets['toxic'], test_size = 0.2)

corpus_train = features_train.values.astype('U')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

corpus_test = features_test.values.astype('U')
tf_idf_test = count_tf_idf.transform(corpus_test)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1. Здесь все прошло без заминки

Комментарии на замечания:
1. Код подкорректировал

In [11]:
# 7. Создаю признаки на обучающей и тестовой выборках

features_train = tf_idf_train
features_test = tf_idf_test

1. Все сделано

In [85]:
# 8. Логистическая регрессия 

model = LogisticRegression(random_state=12345, multi_class='auto', solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print (f1_score(target_test, predictions))


0.7658062102506547


1. Все здорово. F1 лучше таргета.
2. Посмотрим можно ли получить еще лучше

In [86]:
# 9. Решающее дерево. 

model = DecisionTreeClassifier(random_state=12345, max_depth=7) 
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print (f1_score(target_test, predictions))

0.5709391949757351


1. Результат сильно хуже. Как обычно на решающем дереве.

In [96]:
# 10. Случайный лес. 

warnings.filterwarnings('ignore')

for i in range (1,400,99):
    for j in range (1,400,99):
        model = RandomForestClassifier(random_state=12345, n_estimators=i, max_depth = j)
        model.fit(features_train, target_train) 
        predictions = model.predict(features_test)
        print (f1_score(target_test, predictions), i , j)

0.0 1 1
0.4328657314629259 1 100
0.5425975939111221 1 199
0.6275579809004093 1 298
0.6373137647320436 1 397
0.0 100 1
0.40541411537222044 100 100
0.69375 100 199
0.7503628447024673 100 298
0.7640820174404903 100 397
0.0 199 1
0.4115569823434992 199 100
0.7022544700699662 199 199
0.7493940862821135 199 298
0.7645808736717827 199 397
0.0 298 1
0.40541411537222044 298 100
0.6975051975051975 298 199
0.7509113001215066 298 298
0.7637739418302198 298 397
0.0 397 1
0.4018087855297158 397 100
0.6992715920915713 397 199
0.7497575169738118 397 298
0.7638428774254614 397 397


1. Вот это разочарование. Модель отказывается обучаться.

New:
После изменения векторизации ничего не поменялось

In [ ]:
# 10. CatBoost

model = CatBoostClassifier(iterations=200, depth = 16)
model.fit(features_train, target_train, verbose=1) 
predictions = model.predict(features_test)

print (f1_score(target_test, predictions))


Learning rate set to 0.243461


1. Catboost таргет выполнил, но проиграл Логистической регрессии

In [12]:
# 11. И, наконец, - LGBM

for q in range (1,100,15):
    for z in range (1,300,120):
        for w in range (30,300,120):       
            model = LGBMClassifier(loss_function='RMSE', n_iterations = z, max_depth = q, num_leaves = w, min_data_in_leaf = 3)
            model.fit(features_train, target_train)
            predictions = model.predict(features_test)
            print (f1_score(target_test, predictions), q, z, w)


0.46399019307385847 1 1 30
0.46399019307385847 1 1 150
0.46399019307385847 1 1 270
0.46399019307385847 1 121 30
0.46399019307385847 1 121 150
0.46399019307385847 1 121 270
0.46399019307385847 1 241 30
0.46399019307385847 1 241 150
0.46399019307385847 1 241 270
0.7453714835296945 16 1 30
0.7482667941668659 16 1 150
0.7499404052443386 16 1 270
0.7453714835296945 16 121 30
0.7482667941668659 16 121 150
0.7499404052443386 16 121 270
0.7453714835296945 16 241 30
0.7482667941668659 16 241 150
0.7499404052443386 16 241 270
0.761028544468035 31 1 30
0.7719054242002781 31 1 150
0.7614189659169952 31 1 270
0.761028544468035 31 121 30
0.7719054242002781 31 121 150
0.7614189659169952 31 121 270
0.761028544468035 31 241 30
0.7719054242002781 31 241 150
0.7614189659169952 31 241 270
0.761028544468035 46 1 30
0.7801321485532012 46 1 150
0.7749144811858609 46 1 270
0.761028544468035 46 121 30
0.7801321485532012 46 121 150
0.7749144811858609 46 121 270
0.761028544468035 46 241 30
0.7801321485532012 46 

Чуть чуть до таргета не дотянули

In [ ]:
# 12. Bert
## к сожалению, на маленьком количестве сэмплов - F1 неудовлетворительный, при прибавлении kernel - зависает

df_tweets = df_tweets.sample(n = 15000)


tokenizer = transformers.BertTokenizer(
    vocab_file='/datasets/ds_bert/vocab.txt')

tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

config = transformers.BertConfig.from_json_file(
    '/datasets/ds_bert/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '/datasets/ds_bert/rubert_model.bin', config=config)


batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask = attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
        
features = np.concatenate(embeddings)
target = df_tweets['toxic']
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size = 0.5)

model = LogisticRegression(random_state=12345, multi_class='auto', solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print (f1_score(target_test, predictions))


cat_features = features
model = CatBoostClassifier(iterations=100, depth = 7)
model.fit(features_train, target_train, verbose=5) 
predictions = model.predict(features_test)
print (f1_score(target_test, predictions))

model = RandomForestClassifier(random_state=12345, n_estimators=199, max_depth = 397)
model.fit(features_train, target_train) 
predictions = model.predict(features_test)
print (f1_score(target_test, predictions))


# 3. Выводы

Удалось обучить модель с F1, равным почти 0.79.
Задание выполнено.
Победила Логистическая Регрессия.
В целом по проекту все понятно, за исключением неожиданно плохих результатов Random Forest

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны